# Fibril Generator and CyRSoXS Simulation

In [ ]:
from scipy.interpolate import interp1d
from scipy.ndimage import rotate
from sklearn.preprocessing import normalize
from scipy.ndimage import gaussian_filter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import trimesh
import pyembree
import random
import pickle
from NRSS.writer import write_materials, write_hdf5, write_config, write_slurm
from NRSS.checkH5 import checkH5

from Morphology import Morphology
from Fibril import Fibril
import sys
import pathlib


import subprocess
import h5py

from PyHyperScattering.load import cyrsoxsLoader
from PyHyperScattering.integrate import WPIntegrator


import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from matplotlib import cm
from matplotlib.colors import LogNorm

import numpy as np
import io
from PIL import Image
import PIL


%load_ext autoreload
%autoreload 2
%matplotlib inline

## Define path to NRSS and PyHyperScattering

In [ ]:
sys.path.append('/home/devon/Documents/Github/NRSS/')
sys.path.append('/home/devon/Documents/Github/PyHyperScattering/src/')

## Define and generate fibril morphology

In [ ]:
# Declare model box size in nm (x,y,z)
x_dim_nm  = 1024
y_dim_nm  = 1024
z_dim_nm  = 128
pitch_nm = 2 # Dimension of voxel in nm

# Initialize morphology
morphology = Morphology(x_dim_nm, y_dim_nm, z_dim_nm, pitch_nm, 2)
morphology.set_model_parameters(radius_nm_avg = 12,
                                radius_nm_std = 3,
                                max_num_fibrils = 50,
                                fibril_length_range_nm = [100, 400])

morphology.fill_model()

In [ ]:
# May not show if the morphology is too large (too many fibrils)
scene = morphology.get_scene(show_bounding_box=True)
scene.show()

In [ ]:
morphology.voxelize_model()

## Save voxelized morphology as reduced morphology

In [ ]:
from ReducedMorphology import ReducedMorphology
rm = ReducedMorphology(morphology)

In [ ]:
filename = 'test.pickle'
with open(filename, 'wb') as f:
    pickle.dump(rm, f)

## Load voxelized reduced morphology

In [ ]:
rm = None
with open('test.pickle', 'rb') as f:
    rm = pickle.load(f)

## Create material matricies

In [ ]:
# Generate material matricies
mat_Vfrac, mat_S, mat_theta, mat_psi = rm.generate_material_matricies()

# Change material 1 matricies
f2 = 0.90
mat_Vfrac[0,:,:,:] = f2*np.ones_like(mat_Vfrac[0,:,:,:]) + (1-f2)*mat_Vfrac[0,:,:,:]
mat_Vfrac[0,:,:,:] = gaussian_filter(mat_Vfrac[0,:,:,:],  0)
mat_S[0,:,:,:]     = gaussian_filter(0.75*mat_S[0,:,:,:], 0)
mat_theta[0,:,:,:] = gaussian_filter(mat_theta[0,:,:,:],  0)
mat_psi[0,:,:,:]   = gaussian_filter(mat_psi[0,:,:,:],    0)

# Change material 2 matricies
mat_Vfrac[1,:,:,:] = 1 - mat_Vfrac[0,:,:,:]
mat_S[1,:,:,:]     = mat_S[1,:,:,:]
mat_theta[1,:,:,:] = mat_theta[1,:,:,:]
mat_psi[1,:,:,:]   = mat_psi[1,:,:,:]

## Generate simulation input files

In [ ]:
phys_size = 2.0
write_hdf5([[mat_Vfrac[0,:,:,:], mat_S[0,:,:,:], mat_theta[0,:,:,:], mat_psi[0,:,:,:]], 
            [mat_Vfrac[1,:,:,:], mat_S[1,:,:,:], mat_theta[1,:,:,:], mat_psi[1,:,:,:]]],
            phys_size, 'Fibril.hdf5')

In [ ]:
checkH5('Fibril.hdf5', z_slice=25, plotstyle='dark')

In [ ]:
energies = np.round(np.arange(280., 300., 0.5),1)
energies

In [ ]:
material_dict = {'Material1':'P3HT.txt','Material2':'vacuum'}
energy_dict = {'Energy':6,'DeltaPerp':3, 'BetaPerp':1, 'DeltaPara':2, 'BetaPara':0}  
write_materials(energies, material_dict, energy_dict, 2)

In [ ]:
write_config(list(energies), [0.0, 1.0, 360.0], CaseType=0, MorphologyType=0)

## Run CyRSoXS

In [ ]:
subprocess.run(['CyRSoXS','Fibril.hdf5'])

## Load CyRSoXS Results

In [ ]:
basePath = pathlib.Path('.').absolute()
h5path = pathlib.Path(basePath,'HDF5')
h5list = list(sorted(h5path.glob('*h5')))

In [ ]:
def print_key(f, key):
    try:
        keys2 = f[key].keys()
        for key2 in keys2:
            new_key = key + '/' + key2
            print_key(f, new_key)
    except AttributeError:
        print(key)

with h5py.File(h5list[0],'r') as f:
    for key in f.keys():
        print_key(f, key)

In [ ]:
load = cyrsoxsLoader()
integ = WPIntegrator(force_np_backend=True) # avoiding gpu backend for this tutorial

In [ ]:
raw = load.loadDirectory(basePath)

In [ ]:
remeshed = integ.integrateImageStack(raw)

## Visualize CyRSoXS Results

In [ ]:
c = cm.jet(np.linspace(0,1,len(remeshed)))

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3),dpi=140,constrained_layout=True)
raw.sel(energy=283.5).plot(norm=LogNorm(1e-2,1e7),cmap='terrain',ax=ax[0],add_colorbar=False)
raw.sel(energy=285).plot(norm=LogNorm(1e-2,1e7),cmap='terrain',ax=ax[1],add_colorbar=False)
raw.sel(energy=286.5).plot(norm=LogNorm(1e-2,1e7),cmap='terrain',ax=ax[2])

[{axes.set_xlim(-0.1,0.1),axes.set_ylim(-0.1,0.1)} for axes in ax]
plt.show()

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3),dpi=140,constrained_layout=True)
raw.sel(energy=280).plot(norm=LogNorm(1e-2,1e7),cmap='terrain',ax=ax[0],add_colorbar=False)
raw.sel(energy=290).plot(norm=LogNorm(1e-2,1e7),cmap='terrain',ax=ax[1],add_colorbar=False)
raw.sel(energy=299).plot(norm=LogNorm(1e-2,1e7),cmap='terrain',ax=ax[2])

[{axes.set_xlim(-0.4,0.4),axes.set_ylim(-0.4,0.4)} for axes in ax]
plt.show()

In [ ]:
# calculate the anisotropy metric
A = remeshed.rsoxs.AR(chi_width=20)

In [ ]:
A.plot(x='q',cmap='bwr', vmin=-0.45, vmax=0.45)
plt.xlim(left=0.01, right=0.1)
plt.ylim(bottom=280, top=295)
plt.xscale('linear')
plt.show()

In [ ]:
A.plot(x='q',cmap='bwr')
plt.xlim(left=0.01, right=0.1)
plt.ylim(bottom=280, top=295)
plt.xscale('linear')
plt.show()

In [ ]:
A.plot(x='q',cmap='bwr', vmin=-0.45, vmax=0.45)
plt.xlim(left=0.01, right=1)
plt.ylim(bottom=280, top=295)
plt.xscale('linear')
plt.show()

In [ ]:
A.plot(x='q',cmap='bwr', vmin=-0.45, vmax=0.45)
plt.xlim(left=0.01, right=1)
plt.ylim(bottom=280, top=295)
plt.xscale('linear')
plt.show()

In [ ]:
horz = remeshed.rsoxs.slice_chi(0, chi_width=20)
vert = remeshed.rsoxs.slice_chi(90, chi_width=20)

In [ ]:
vert.plot(x='q',cmap='terrain',norm=LogNorm(5e1,1e5), xscale='linear')
plt.xlim(left=0.01, right=0.1)
plt.show()

In [ ]:
vert.plot(x='q',cmap='terrain',norm=LogNorm(1e-1,1e7), xscale='linear')
plt.xlim(left=0.01, right=1)
plt.show()

In [ ]:
vert.sel(energy=280).plot(yscale='log',xscale='linear')
plt.xlim(left=0.01, right=1)
plt.ylim(bottom=1e-5, top=1e2)
plt.show()